# Ejercicio 12: Multimodal Embeddings

## Objetivo de la práctica

El objetivo de este ejercicio es observar cómo modelos multimodales como CLIP llevan texto e imágenes al mismo espacio vectorial, y verificar graficando embeddings en 2D.

### Pasos:

1. Obtener embeddings de imágenes y textos con CLIP.
2. Mostrar que ambos viven en el mismo espacio (misma dimensión y comparables).
3. Proyectar los vectores a 2D (PCA / t-SNE / UMAP) y graficarlos en un plano.
4. Verificar emparejamientos usando cosine similarity (búsqueda texto→imagen e imagen→texto).

In [ ]:
pip install git+https://github.com/openai/CLIP.git


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.metrics.pairwise import cosine_similarity
from transformers import CLIPProcessor, CLIPModel

import glob
import clip
import os

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print("Dispositivo:", device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# CARGAR MODELO CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
# 1. CARGAR IMÁGENES
CARPETA_IMAGENES = "/content/drive/MyDrive/RI/RI"

rutas_imagenes = sorted(glob.glob(os.path.join(CARPETA_IMAGENES, "*.jpg")))
imagenes = [Image.open(ruta).convert("RGB") for ruta in rutas_imagenes]

print(f"Imágenes cargadas: {len(imagenes)}")

In [ ]:
# 2. PREPROCESAR IMÁGENES PARA CLIP
imagenes = [
    Image.open(p).convert("RGB")
    for p in rutas_imagenes
]


In [ ]:
# 3. OBTENER EMBEDDINGS DE IMÁGENES
inputs = processor(images=imagenes, return_tensors="pt")

with torch.no_grad():
    emb_imagenes = model.get_image_features(**inputs)

# Normalizar embeddings (norma = 1)
emb_imagenes = emb_imagenes / emb_imagenes.norm(dim=-1, keepdim=True)

# Pasar a numpy (para PCA, t-SNE, cosine similarity)
emb_imagenes = emb_imagenes.numpy()

print("Embeddings de imágenes:", emb_imagenes.shape)

In [ ]:
texts = [
    "dog",
    "cat",
    "car",
    "bike",
    "person ",
    "bed"
]

inputs = processor(text=texts, return_tensors="pt", padding=True)

with torch.no_grad():
    emb_textos = model.get_text_features(**inputs)

# Normalizar embeddings
emb_textos = emb_textos / emb_textos.norm(dim=-1, keepdim=True)

# Pasar a numpy (para PCA, t-SNE, cosine similarity)
emb_textos = emb_textos.numpy()

print("Embeddings de texto:", emb_textos.shape)

In [ ]:
print("Dimensión imagen:", emb_imagenes.shape[1])
print("Dimensión texto:", emb_textos.shape[1])


In [ ]:
similarity = cosine_similarity(
    emb_textos,
    emb_imagenes
)

similarity


In [ ]:
# UNIR EMBEDDINGS (imagen + texto)
all_embeddings = np.vstack([emb_imagenes, emb_textos])

# PCA 2D
pca = PCA(n_components=2)
embeddings_2d_pca = pca.fit_transform(all_embeddings)

img_2d_pca = embeddings_2d_pca[:len(emb_imagenes)]
txt_2d_pca = embeddings_2d_pca[len(emb_imagenes):]

# t-SNE 2D
tsne = TSNE(n_components=2, perplexity=12, random_state=42)
embeddings_2d_tsne = tsne.fit_transform(all_embeddings)

img_2d_tsne = embeddings_2d_tsne[:len(emb_imagenes)]
txt_2d_tsne = embeddings_2d_tsne[len(emb_imagenes):]

In [ ]:
plt.figure(figsize=(9,7))

plt.scatter(img_2d_tsne[:, 0], img_2d_tsne[:, 1], marker='o', label='Imágenes')
for i, name in enumerate(rutas_imagenes):
    plt.text(
        img_2d_tsne[i, 0],
        img_2d_tsne[i, 1],
        os.path.basename(name),
        fontsize=8
    )

plt.scatter(txt_2d_tsne[:, 0], txt_2d_tsne[:, 1], marker='x', label='Textos')
for i, txt in enumerate(texts):
    plt.text(
        txt_2d_tsne[i, 0],
        txt_2d_tsne[i, 1],
        txt,
        fontsize=10
    )

plt.title("Embeddings Multimodales CLIP (t-SNE 2D)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(9,7))

plt.scatter(img_2d_pca[:, 0], img_2d_pca[:, 1], marker='o', label='Imágenes')
for i, name in enumerate(rutas_imagenes):
    plt.text(
        img_2d_pca[i, 0],
        img_2d_pca[i, 1],
        os.path.basename(name),
        fontsize=8
    )

plt.scatter(txt_2d_pca[:, 0], txt_2d_pca[:, 1], marker='x', label='Textos')
for i, txt in enumerate(texts):
    plt.text(
        txt_2d_pca[i, 0],
        txt_2d_pca[i, 1],
        txt,
        fontsize=10
    )

plt.title("Embeddings Multimodales CLIP (PCA 2D)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def buscar_imagen(texto):
    # Embedding del texto
    inputs = processor(text=[texto], return_tensors="pt", padding=True)
    with torch.no_grad():
        emb_texto = model.get_text_features(**inputs)

    # Normalizar
    emb_texto = emb_texto / emb_texto.norm(dim=-1, keepdim=True)
    emb_texto = emb_texto.numpy()

    # Similaridad coseno texto → imágenes
    scores = cosine_similarity(
        emb_texto,
        emb_imagenes
    )[0]

    idx = np.argmax(scores)

    print("Texto:", texto)
    print("Imagen más similar:", os.path.basename(rutas_imagenes[idx]))
    print("Score:", scores[idx])

    # Mostrar imagen
    img = Image.open(rutas_imagenes[idx])
    plt.imshow(img)
    plt.axis("off")
    plt.show()


In [ ]:
def buscar_texto(nombre_imagen):
    # Cargar imagen
    image = Image.open(nombre_imagen).convert("RGB")

    # Embedding de la imagen
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        emb_imagen = model.get_image_features(**inputs)

    # Normalizar
    emb_imagen = emb_imagen / emb_imagen.norm(dim=-1, keepdim=True)
    emb_imagen = emb_imagen.numpy()

    # Similaridad coseno imagen → textos
    scores = cosine_similarity(
        emb_imagen,
        emb_textos
    )[0]

    # Mostrar scores ordenados
    text_scores = sorted(zip(texts, scores), key=lambda x: x[1], reverse=True)
    print("Scores texto:")
    for t, s in text_scores:
        print(f"{t:35s} -> {s:.4f}")

    idx = np.argmax(scores)

    print("\nImagen:", os.path.basename(nombre_imagen))
    print("Texto más cercano:", texts[idx])

    plt.imshow(image)
    plt.axis("off")
    plt.show()


In [ ]:
buscar_imagen("person 2")

In [ ]:
buscar_texto("/content/drive/MyDrive/RI/RI/person 1.jpg")